# HW 2 - Considering Bias in Data

## Goal
This assignment aims to explore bias in data through the analysis of Wikipedia articles on political figures across different countries.

## Data Sources
1. **Wikipedia Articles**: A dataset of articles on politicians was generated by crawling the [Wikipedia Category:Politicians by nationality](https://en.wikipedia.org/wiki/Category:Politicians_by_nationality). [`politicians_by_country.AUG.2024.csv`](https://drive.google.com/drive/folders/1qINqVxEf072AKY1HpaQYlFTEllLWjl4L).

2. **Population Data**: Population data is sourced from the World Population Data Sheet published by the Population Reference [`population_by_country_AUG.2024.csv`](https://drive.google.com/drive/folders/1qINqVxEf072AKY1HpaQYlFTEllLWjl4L).

## API
**ORES**: The assignment utilizes the ORES machine learning service, now being integrated into Wikimedia's new infrastructure called LiftWing. This transition moves well-established ORES ML models to LiftWing for generating article quality estimates.
The ORES API documentation can be accessed on the [main ORES page](https://www.mediawiki.org/wiki/ORES).

## License
This project is based on a code example developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 15, 2023



In [120]:
import pandas as pd

politicians_by_country = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/politicians_by_country_AUG.2024.csv')
population_by_country = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/population_by_country_AUG.2024.csv')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

In [4]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_HEADER_AGENT = 'User-Agent'

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'jbjunguw@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2024'
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


In [5]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None,
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT,
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    if API_HEADER_AGENT not in headers:
        raise Exception(f"The header data should include a '{API_HEADER_AGENT}' field that contains your UW email address.")

    if 'uwnetid@uw' in headers[API_HEADER_AGENT]:
        raise Exception(f"Use your UW email address in the '{API_HEADER_AGENT}' field.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [6]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

In [11]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT,
                                   model_name = API_ORES_EN_QUALITY_MODEL,
                                   request_data = ORES_REQUEST_DATA_TEMPLATE,
                                   header_format = REQUEST_HEADER_TEMPLATE,
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):

    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token

    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")

    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)

    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [17]:
!pip install tqdm

In [21]:
import math

def chunk_list(data, n):
    for i in range(0, len(data), n):
        yield data[i:i + n]

politicians_dict = politicians_by_country.groupby('country').apply(lambda df: df.to_dict('records')).to_dict()

no_matching = []
matching = []
missing = []


<ipython-input-21-3f025d748965>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  politicians_dict = politicians_by_country.groupby('country').apply(lambda df: df.to_dict('records')).to_dict()


In [19]:
from tqdm import tqdm

politicians_dict = politicians_by_country.groupby('country').apply(lambda df: df.to_dict('records')).to_dict()

no_matching = []
matching = []
missing = []

<ipython-input-19-dde980a5afd5>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  politicians_dict = politicians_by_country.groupby('country').apply(lambda df: df.to_dict('records')).to_dict()


In [33]:
for index, row in tqdm(population_by_country.iterrows(), total=population_by_country.shape[0]):
    geography = row["Geography"]
    print(f"Processing country/region: {geography}")

    if geography.upper() == geography:
        cur_region = geography
    else:
        if geography not in politicians_dict:
            no_matching.append(geography)
        else:
            politicians = politicians_dict[geography]
            politician_names = [politician['name'] for politician in politicians]

            # Split the names into chunks of 50 (API limit)
            for chunk in chunk_list(politician_names, 50):
                politician_titles = "|".join(chunk)

                request_info = PAGEINFO_PARAMS_TEMPLATE.copy()
                request_info['titles'] = politician_titles

                pageinfo_response = request_pageinfo_per_article(request_template=request_info)

                if 'query' in pageinfo_response and 'pages' in pageinfo_response['query']:
                    pageinfo_responses = pageinfo_response['query']['pages']
                else:
                    print(f"Unexpected response format or error: {json.dumps(pageinfo_response, indent=4)}")
                    missing.append(f"Error retrieving pageinfo for {politician_titles}")
                    continue

                # Process each politician in the chunk
                for politician in chunk:
                    pageinfo = next((p for p in pageinfo_responses.values() if p.get('title') == politician), None)

                    if pageinfo:
                        if 'lastrevid' in pageinfo:
                            lastrevid = pageinfo['lastrevid']

                            score = request_ores_score_per_article(
                                article_revid=lastrevid,
                                email_address="jbjunguw@uw.edu",
                                access_token=ACCESS_TOKEN
                            )

                            if score and 'enwiki' in score and 'scores' in score['enwiki'] and str(lastrevid) in score['enwiki']['scores'] \
                                and 'articlequality' in score['enwiki']['scores'][str(lastrevid)] \
                                and 'prediction' in score['enwiki']['scores'][str(lastrevid)]['articlequality']['score']:

                                article_quality = score['enwiki']['scores'][str(lastrevid)]['articlequality']['score']['prediction']
                                result = [
                                    ('country', geography),
                                    ('region', cur_region),
                                    ('population', row['Population'] * 10000),
                                    ('article_title', politician),
                                    ('revision_id', lastrevid),
                                    ('article_quality', article_quality)
                                ]
                                matching.append(result)
                            else:
                                missing.append(politician)
                        else:
                            print(f"Missing 'lastrevid' for {politician}")
                            missing.append(politician)
                    else:
                        print(f"Missing page info for {politician}")
                        missing.append(politician)


  0%|          | 0/233 [00:00<?, ?it/s]

Processing country/region: WORLD
Processing country/region: AFRICA
Processing country/region: NORTHERN AFRICA
Processing country/region: Algeria


  2%|▏         | 4/233 [01:16<1:13:19, 19.21s/it]

Processing country/region: Egypt


  2%|▏         | 5/233 [01:53<1:30:29, 23.81s/it]

...

 99%|█████████▉| 231/233 [2:06:36<00:21, 10.95s/it]

Processing country/region: Tuvalu


100%|█████████▉| 232/233 [2:06:38<00:09,  9.21s/it]

Processing country/region: Vanuatu


100%|██████████| 233/233 [2:06:43<00:00, 32.63s/it]


The error rate is calculated as the ratio of articles without scores (8) to the total number of articles, indicating the proportion of missing scores in the dataset.

In [92]:
8/len(politicians_by_country) * 100

0.11180992313067784

In [88]:
no_matching

['Reunion',
 'French Guiana',
 'Guadeloupe',
 'Martinique',
 'Brunei',
 'Mauritius',
 'French Polynesia',
 'Mayotte',
 'Australia',
 'Canada',
 'Dominica',
 'eSwatini',
 'Netherlands',
 'Denmark',
 'GuineaBissau',
 'Curacao',
 'Andorra',
 'Suriname',
 'Mexico',
 'Nauru',
 'China (Macao SAR)',
 'Jamaica',
 'Sao Tome and Principe',
 'Fiji',
 'United States',
 'Georgia',
 'Palau',
 'Romania',
 'Iceland',
 'San Marino',
 'Liechtenstein',
 'China (Hong Kong SAR)',
 'United Kingdom',
 'Guam',
 'Kiribati',
 'Korea (South)',
 'New Zealand',
 'Puerto Rico',
 'Ireland',
 'New Caledonia',
 'Philippines',
 'Korea (North)',
 'Western Sahara']

In [59]:
with open('wp_countries-no_match.txt', 'w') as f:
    for item in no_matching:
        f.write("%s\n" % item)

In [61]:
import pandas as pd

matching_df = pd.DataFrame(
    [{k: v for k, v in row} for row in matching]
)

matching_df = matching_df[
    ["country", "region", "population", "article_title", "revision_id", "article_quality"]
]

unique_matching_df = matching_df.drop_duplicates(subset=["article_title", "revision_id"])

unique_matching_df.to_csv("wp_politicians_by_country.csv", index=False)

In [111]:
population_by_country = population_by_country.set_index("Geography")

article_counts = matching_df.groupby("country")["article_title"].count()

article_coverage = article_counts / population_by_country.loc[article_counts.index, "Population"]

analysis_df = pd.DataFrame(article_coverage, columns=["article_coverage"])

quality_counts = (
    matching_df[
        (matching_df["article_quality"] == "FA") | (matching_df["article_quality"] == "GA")
    ]
    .groupby("country")["article_title"]
    .count()
)

quality_coverage = quality_counts / population_by_country.loc[quality_counts.index, "Population"]

analysis_df["quality_coverage"] = quality_coverage

analysis_df.fillna(0, inplace=True)

top_10_article_coverage = analysis_df.sort_values("article_coverage", ascending=False).head(10)
bottom_10_article_coverage = analysis_df.sort_values("article_coverage").head(10)

top_10_quality_coverage = analysis_df.sort_values("quality_coverage", ascending=False).head(10)
bottom_10_quality_coverage = analysis_df.sort_values("quality_coverage").head(10)

In [112]:
print("Top 10 countries by article coverage:")
top_10_article_coverage[:]["article_coverage"]

Top 10 countries by article coverage:


,article_coverage
country,
Monaco,inf
Tuvalu,inf
Antigua and Barbuda,330.000000
Federated States of Micronesia,140.000000
Marshall Islands,130.000000
Tonga,100.000000
Barbados,83.333333
Seychelles,60.000000
Montenegro,60.000000


In [113]:
print("Bottom 10 countries by article coverage:")
bottom_10_article_coverage[:]["article_coverage"]

Bottom 10 countries by article coverage:


,article_coverage
country,
China,0.011337
Ghana,0.087977
India,0.105698
Saudi Arabia,0.135501
Zambia,0.148515
Norway,0.181818
Israel,0.204082
Egypt,0.304183
Cote d'Ivoire,0.323625


In [114]:
print("Top 10 countries by quality coverage:")
top_10_quality_coverage[:]["quality_coverage"]

Top 10 countries by quality coverage:


,quality_coverage
country,
Montenegro,5.000000
Luxembourg,2.857143
Albania,2.592593
Kosovo,2.352941
Maldives,1.666667
Lithuania,1.379310
Croatia,1.315789
Guyana,1.250000
Palestinian Territory,1.090909


In [115]:
print("Bottom 10 countries by quality coverage:")
bottom_10_quality_coverage[:]["quality_coverage"]

Bottom 10 countries by quality coverage:


,quality_coverage
country,
Lesotho,0.0
Niger,0.0
Nicaragua,0.0
Namibia,0.0
Mozambique,0.0
Monaco,0.0
Marshall Islands,0.0
Malta,0.0
Malaysia,0.0


In [68]:
region_article_coverage = matching_df.groupby("region")["article_title"].count()
region_article_coverage

,article_title
region,
CARIBBEAN,217
CENTRAL AMERICA,188
CENTRAL ASIA,106
EAST ASIA,152
EASTERN AFRICA,662
EASTERN EUROPE,709
MIDDLE AFRICA,230
NORTHERN AFRICA,401
NORTHERN EUROPE,191


In [78]:
region_article_coverage_df = pd.DataFrame(region_article_coverage).reset_index()
region_population_df = population_by_country.rename(columns={"Geography": "region"})
merged_df = pd.merge(region_article_coverage_df, region_population_df, on='region', how='left')

In [79]:
merged_df['article_coverage_per_capita'] = merged_df['article_title'] / merged_df['Population']

ranked_regions_total_coverage = merged_df[['region', 'article_coverage_per_capita']].sort_values(
    by='article_coverage_per_capita', ascending=False
).set_index('region')

ranked_regions_total_coverage

,article_coverage_per_capita
region,
SOUTHERN EUROPE,5.230263
CARIBBEAN,4.931818
WESTERN EUROPE,2.497487
EASTERN EUROPE,2.487719
WESTERN ASIA,2.036789
NORTHERN EUROPE,1.768519
SOUTHERN AFRICA,1.757143
OCEANIA,1.600000
NORTHERN AFRICA,1.566406


In [117]:
high_quality_articles = matching_df[matching_df["article_quality"].isin(["FA", "GA"])]
region_high_quality_coverage = high_quality_articles.groupby("region")["article_title"].count()

region_high_quality_coverage_df = pd.DataFrame(region_high_quality_coverage).reset_index()
quality_merged_df = pd.merge(region_high_quality_coverage_df, region_population_df, on='region', how='left')

quality_merged_df['quality_coverage_per_capita'] = quality_merged_df['article_title'] / quality_merged_df['Population']

ranked_regions_total_quality_coverage = quality_merged_df[['region', 'quality_coverage_per_capita']].sort_values(
    by='quality_coverage_per_capita', ascending=False
).set_index('region')

ranked_regions_total_quality_coverage

,quality_coverage_per_capita
region,
SOUTHERN EUROPE,0.348684
CARIBBEAN,0.204545
EASTERN EUROPE,0.133333
SOUTHERN AFRICA,0.114286
WESTERN EUROPE,0.105528
WESTERN ASIA,0.090301
NORTHERN EUROPE,0.083333
NORTHERN AFRICA,0.070312
CENTRAL ASIA,0.062500
